## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-27-14-52-39 +0000,bbc,Firefighters search for hundreds still missing...,https://www.bbc.com/news/articles/c62n3enz9klo...
1,2025-11-27-14-52-03 +0000,nyt,"Faith Winter, Colorado State Senator, Is Kille...",https://www.nytimes.com/2025/11/27/us/faith-wi...
2,2025-11-27-14-51-42 +0000,nyt,Hong Kong’s Worst Fire in Decades Fuels Scruti...,https://www.nytimes.com/2025/11/27/world/asia/...
3,2025-11-27-14-50-44 +0000,wapo,D.C. shooting live updates: National Guard sho...,https://www.washingtonpost.com/dc-md-va/2025/1...
4,2025-11-27-14-50-44 +0000,wapo,U.S. Attorney Jeanine Pirro said Thursday she ...,https://www.washingtonpost.com


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,35
26,national,24
9,fire,24
5,hong,22
6,kong,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
19,2025-11-27-14-19-01 +0000,startribune,Afghan national in custody after shooting of 2...,https://www.startribune.com/afghan-national-in...,109
218,2025-11-26-20-58-29 +0000,cbc,FBI to investigate motive in D.C. shooting as ...,https://www.cbc.ca/news/world/trump-afghanista...,108
202,2025-11-26-21-59-46 +0000,nypost,Trump requests 500 more National Guard troops ...,https://nypost.com/2025/11/26/us-news/trump-re...,106
138,2025-11-27-04-06-00 +0000,wsj,Trump Calls D.C. National Guard Shooting ‘Act ...,https://www.wsj.com/us-news/national-guard-sho...,101
236,2025-11-26-19-58-01 +0000,nyt,Hong Kong Fire Shared Similarities With Grenfe...,https://www.nytimes.com/2025/11/26/world/asia/...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
19,109,2025-11-27-14-19-01 +0000,startribune,Afghan national in custody after shooting of 2...,https://www.startribune.com/afghan-national-in...
236,100,2025-11-26-19-58-01 +0000,nyt,Hong Kong Fire Shared Similarities With Grenfe...,https://www.nytimes.com/2025/11/26/world/asia/...
306,88,2025-11-26-16-23-23 +0000,cbc,New prosecutor won't pursue charges against Tr...,https://www.cbc.ca/news/world/trump-georgia-el...
243,52,2025-11-26-19-26-47 +0000,nypost,Pope Leo XIV embarking on first foreign trip t...,https://nypost.com/2025/11/26/world-news/pope-...
226,52,2025-11-26-20-33-00 +0000,wsj,The job market continues to sputter even as co...,https://www.wsj.com/economy/central-banking/fe...
70,47,2025-11-27-11-01-00 +0000,nypost,Meet NYC’s hardest workers on Thanksgiving — l...,https://nypost.com/2025/11/27/lifestyle/nycs-h...
23,42,2025-11-27-14-14-31 +0000,nyt,Trump Cut Europe Out of Ukraine Talks. Here’s ...,https://www.nytimes.com/2025/11/27/world/europ...
81,38,2025-11-27-10-40-00 +0000,wsj,Silver Prices Rise as China’s Stockpile Hits 1...,https://www.wsj.com/finance/commodities-future...
38,37,2025-11-27-13-17-00 +0000,wsj,"Centrist dealmaker Sen. Jeanne Shaheen (D., N....",https://www.wsj.com/politics/policy/jeanne-sha...
63,34,2025-11-27-11-38-39 +0000,bbc,"'I earn £20,000 and live with my son. The Budg...",https://www.bbc.com/news/articles/c9d6zwppjvjo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
